In [2]:
from langchain_community.document_loaders import YoutubeLoader

video_id = "5MgBikgcWnY"  # TEDx video
docs = []
try:
    
    loader = YoutubeLoader.from_youtube_url(
        "https://www.youtube.com/watch?v=QsYGlZkevEg", add_video_info=False
    )
    docs = loader.load()
    print(docs)

except Exception as e:
    print("❌ Failed to load transcript.")
    print(type(e).__name__ + ":", e)


❌ Failed to load transcript.
ParseError: no element found: line 1, column 0


In [17]:
from huggingface_hub import InferenceClient
import os
from dotenv import load_dotenv
load_dotenv()
# Paste your HF token here or set via environment variable
token = os.getenv('HF_TOKEN')

# Pick a chat-supported model
model_id = "HuggingFaceH4/zephyr-7b-beta"  # Example

client = InferenceClient(model=model_id, token=token)

# OpenAI-style chat message format
messages = [
    {"role": "system", "content": "You are a helpful coding assistant."},
    {"role": "user", "content": "Write a Python function to reverse a list."}
]

# Call chat_completion
response = client.chat_completion(
    messages=messages,
    max_tokens=200,
    temperature=0.3
)

print("✅ Chat Response:\n", response.choices[0].message["content"])


✅ Chat Response:
 Here's a Python function to reverse a list:

```python
def reverse_list(lst):
    """
    This function takes a list as input and returns the reversed list.
    """
    return list(reversed(lst))
```

Explanation:

The `reversed()` function returns a reverse iterator for a sequence. We pass our list `lst` to this function and convert the resulting reverse iterator to a list using the `list()` function. This list is then returned by the function.

Example usage:

```python
my_list = [1, 2, 3, 4, 5]
reversed_list = reverse_list(my_list)
print(reversed_list)  # Output: [5, 4, 3, 2, 1]
```
